# 海上风电场

等级：入门

## 目的和先决条件

在此示例中，我们将解决如何最小化铺设水下电缆以收集海上风电场产生的电力的成本的问题。我们将构建此问题的混合整数编程（MIP）模型，在Gurobi Python界面中实现此模型，并计算最佳解决方案。

该建模示例处于初级阶段，我们假设您了解Python，并且具有一些有关构建数学优化模型的知识。

**注意：** 您可以通过单击 [此处](https://github.com/Gurobi/modeling-examples/archive/master.zip) 下载包含此示例和其他示例的仓库。为了正确运行此 Jupyter Notebook，您必须具有Gurobi许可证。如果您没有，则可以 *商业用户身份* 申请
[评估许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=CommercialDataScience)，或以 *学术用户* 的身份下载
[免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=AcademicDataScience)。

## 动机

全球气候变化已经对环境产生了明显影响。冰川消退，河流和湖泊上的冰块破裂早于预期，动植物物种受到影响，树木开花快于预期。全球气候变化的潜在未来影响包括更频繁的山货，某些地区更长的干旱时期以及热带风暴的数量，持续时间和强度增加。[1]

缓解气候变化包括限制全球变暖的幅度或速度及其相关影响的行动。缓解气候变化的第一个挑战是消除煤炭，石油和最终天然气的燃烧。这可能是最艰巨的挑战，因为富裕国家的居民吃饭、穿衣、工作、娱乐甚至睡觉都离不开化石燃料。同样，发展中国家的人民也希望且也应该得到同样的生活。目前对于减少对化石燃料的依赖，并没有完美的解决方案（例如，碳中性生物燃料会抬高食品价格并导致森林破坏；核能虽然不会排放温室气体，但会产生放射性废物）。其他替代品还包括植物衍生的塑料，生物柴油和风力发电等。

海上风力发电是利用风力发电场建造在水体中，通常是在海洋中，以收集风能来发电。与陆地相比，海上的风速更高，因此海上风力发电的单位装机容量更高。

将风力涡轮机安装在海上的好处是，沿海地区的风力要强得多，与大陆上的风力不同，下午的海上微风可能很强劲，与人们用电最多的时间相称。海上涡轮机也可以靠近沿海地区（例如大城市）的负载中心，从而无需新的长距离传输线。


## 问题描述

海上风电场是一组放置在海上的风力涡轮机，以利用强大的海上风力。这些强风可以产生更多的电力，但海上风力发电场的安装和运营成本比陆地上的要高。

我们将使用MIP模型来减少建造海上风电场的部分成本。我们将计算出如何铺设连接涡轮机的水下电缆的计划。这些电缆是将涡轮机产生的电力输送到陆地所必需的。我们计算的计划将使安装水下电缆的成本降到最低，同时确保每个涡轮机都与海岸相连，每根电缆都有足够的容量来处理产生的电流。

在我们的例子中，丹麦西海岸正在建设一个风力发电场。海岸上有一座发电站，所有的电力都必须被输送到电网中。风电场中还设有中转站，可以收集来自多个涡轮机的电力，并通过一条电缆将其传送到岸上。

在安装电缆时，我们必须考虑两个因素。首先，在海底铺设电缆是有固定成本的。这种成本与电缆连接的两个站点之间的距离成正比。第二，我们必须考虑有多少电流会通过电缆。承载大电流的连接需要厚电缆。粗电缆比细电缆贵。

该优化问题的目标是以最小的成本铺设连接风电场电网的电缆。

海上风电场优化问题模型是一种更为普遍的优化模型（称为固定费用和流量问题）一个实例。固定收费网络流量问题可以应用于大量的业务问题——例如，在通信和运输网络的规划中。


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation for the offshore wind farming problem.

## Model Formulation

### Sets and Indices

$G(V,E)$: Graph that represents the wind farm network, where $V$ is the set of vertices and $E$ is the set of edges. The turbines, transfer stations, and power stations are vertices in the set of vertices $V$ of the graph. The set of potential cables are the edges of the graph. 

### Parameters

$s_{i} \in \mathbb{R}$: Power supplied at vertex $i \in V$. Since turbines supply power,  they are source vertices with $s_{i} > 0$. Transfer stations do not supply or remove power from the network so they have $s_{i} = 0$. The power station on the coast is a sink that remove all power from the wind farm so it has $s_{i} < 0$.

$u_{i,j} \in \mathbb{R}^+ $: Maximum current capacity a cable can handle from vertex $i \in V$ to vertex $j \in V$.

$c_{i,j} \in \mathbb{R}^+$: Cost per unit of current flow going from vertex $i \in V$ to vertex $j \in V$, i.e. the price we must pay to increase the thickness of the cable to handle an increase in current. 

$f_{i,j} \in \mathbb{R}^+$: Fixed cost of laying a cable from vertex $i \in V$ to vertex $j \in V$, and is the result of multiplying the distance between vertices by the cost per mile.

### Decision Variables

$install_{i,j} \in \{0, 1 \}$: This variable is equal to 1 if we lay a cable from vertex $i \in V$ to vertex $j \in V$; and 0 otherwise.

$flow_{i,j} \geq 0$: This non-negative continuous variable represents the amount of current flowing from vertex $i \in V$ to vertex $j \in V$.

The goal of this optimization model is to decide which of these potential edges in the graph should be used at a minimum cost.

### Objective Function

- **Total costs**. We want to minimize the total cost to install the cables. The term on the left is the variable costs (i.e. those that vary according to the current in the cable). The term on right is the fixed cost to install the cable.

\begin{equation}
\text{Max} \quad Z = \sum_{(i,j) \in E} c_{i,j} \cdot flow_{i,j} + \sum_{(i,j) \in E} f_{i,j} \cdot install_{i,j} 
\tag{0}
\end{equation}

### Constraints

- **Balance**. For each vertex $i \in V$, we want to ensure the conservation of current in the network.

\begin{equation}
\sum_{j:(i,j) \in E} flow_{i,j} - \sum_{j:(j,i) \in E} flow_{j,i} = s_{i} \quad \forall i \in V
\tag{1}
\end{equation}

- **Capacity**. For each edge $(i,j) \in E$, we want to enforce the limits on the maximum current capacity of each cable.

\begin{equation}
0 \leq flow_{i,j} \leq u_{i,j} \cdot install_{i,j}  \quad \forall (i,j) \in E
\tag{2}
\end{equation}

## Python Implementation

This example considers three turbines, one transfer station, and two power stations. The current flowing out at each vertex of the wind farm network is presented in the following table. Recall that since turbines supply power their capacity is positive. Transfer stations do not supply or remove power from the network so they have a capacity of zero.  The power stations on the coast are sinks that remove all power from the wind farm network so they have demand of power, in this case we use a negative number.

| <i></i> | Capacity in MW |  
| --- | --- | 
| vertex 1 | 4 |
| vertex 2 | 3 |
| vertex 3 | 2 |
| vertex 4 | 0 |
| vertex 5 | -6 |
| vertex 6 | -3 |


The capacity, flow cost, and fixed cost of each edge in the wind farm network are provided in the following table.

| <i></i> | Capacity in MW  | Flow cost in millions of Euros | Fixed cost in millions of Euros| 
| --- | --- | --- | --- |
| Edge: (0,4) | 4 | 1 | 1 |
| Edge: (0,3) | 2 | 1 | 1 |
| Edge: (1,3) | 3 | 1 | 1 |
| Edge: (2,5) | 2 | 1 | 1 |
| Edge: (3,4) | 2 | 1 | 1 |
| Edge: (3,5) | 1 | 1 | 1 |


We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Parameters

vertices = {0: 4, 1: 3, 2: 2, 3: 0, 4: -6, 5: -3}
edges, cap, flow_cost, fixed_cost = gp.multidict({
    (0,4): [4,1,1],
    (0,3): [2,1,1],
    (1,3): [3,1,1],
    (2,5): [2,1,1],
    (3,4): [2,1,1],
    (3,5): [1,1,1]
})

### Model Deployment

We now determine the MIP model for the offshore wind farming problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to lay cables at the offshore wind farming network that minimizes total costs.

In [2]:
# MIP  model formulation

m = gp.Model("offshore_wind_farming")

# Add variables
install = m.addVars(edges, vtype=GRB.BINARY, name="Install")
flow = m.addVars(edges, vtype=GRB.CONTINUOUS, name="Flow")

# Add constraints
m.addConstrs((flow.sum(v,'*') - flow.sum('*',v) == supply for v, supply in vertices.items()), name="Flow_conservation")
m.addConstrs((flow[e] <= cap[e]*install[e] for e in edges), name="Install2flow")

# Set objective
m.setObjective(flow.prod(flow_cost) + install.prod(fixed_cost), GRB.MINIMIZE)

m.optimize()

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 12 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x03b3989f
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 17 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.700000000000e+01, best bound 1.700000000000e+01, gap 0.0000%


## Analysis


The result of the optimization model shows that the minimum total cost value is 17 million Euros. Let's see the solution that achieves that optimal result.

### Cable Installation Plan

This plan determines the layout of cables in the offshore wind farming network.

In [3]:
# display which edges in the offshore wind farming network we plan to install.

for origin, end in install.keys():
    if (abs(install[origin, end].x) > 0.5):
        print(f"Install cable from location {origin + 1} to location {end + 1} in the offshore wind farming network ")
        


Install cable from location 1 to location 5 in the offshore wind farming network 
Install cable from location 2 to location 4 in the offshore wind farming network 
Install cable from location 3 to location 6 in the offshore wind farming network 
Install cable from location 4 to location 5 in the offshore wind farming network 
Install cable from location 4 to location 6 in the offshore wind farming network 


### Cable Capacity Plan 

This plan determines the current flow capacity in MW of each cable installed.

In [4]:
# Current flow capacity of each cable installed

for origin, end in flow.keys():
    if (abs(flow[origin, end].x) > 1e-6):
         print(f"The capacity of cable installed from location {origin + 1} to location {end + 1} is {flow[origin, end].x} MW ")

The capacity of cable installed from location 1 to location 5 is 4.0 MW 
The capacity of cable installed from location 2 to location 4 is 3.0 MW 
The capacity of cable installed from location 3 to location 6 is 2.0 MW 
The capacity of cable installed from location 4 to location 5 is 2.0 MW 
The capacity of cable installed from location 4 to location 6 is 1.0 MW 


##  Conclusion

In this example, we addressed an offshore wind farming problem where we want to minimize the cost of laying underwater cables to collect electricity produced by an offshore wind farm network. We learned how to formulate the problem as a MIP model. Also, we learned how to implement the MIP model formulation and solve it using the Gurobi Python API.

## References

[1] https://climate.nasa.gov/effects/

[2] https://www.scientificamerican.com/article/10-solutions-for-climate-change/

Copyright © 2020 Gurobi Optimization, LLC